# 📘 Архитектуры Агентов 9: Планирование Tree-of-Thoughts (Дерево Мыслей)

Добро пожаловать в девятый блокнот нашей серии. Сегодня мы исследуем мощную архитектуру рассуждений и планирования: **Tree-of-Thoughts (ToT)**. Этот паттерн поднимает возможности агента по решению проблем от линейной цепочки мыслей до многовариантного исследовательского поиска.

Вместо генерации единственной, последовательной линии рассуждений, агент ToT генерирует несколько кандидатов "мыслей" или следующих шагов на каждом этапе проблемы. Затем он оценивает эти мысли, отсекая невалидные или бесперспективные ветви и расширяя наиболее многообещающие. Это создает дерево поиска, где агент может возвращаться назад, исследовать альтернативы и систематически навигировать по сложному пространству проблемы.

Чтобы продемонстрировать это, мы поставим перед нашим агентом классическую логическую головоломку: **задача о Волке, Козе и Капусте**. Эта головоломка знаменита, потому что она требует неочевидных шагов (например, вернуть предмет *обратно*) и имеет несколько невалидных состояний, которые могут заманить в ловушку наивного решателя. Мы покажем, как простой агент **Chain-of-Thought** может потерпеть неудачу, в то время как агент **ToT** методично строит валидный план, исследуя и оценивая дерево возможностей.

### Определение
**Tree-of-Thoughts (ToT)** — это фреймворк агентного рассуждения, где решение проблем моделируется как поиск по дереву. Агент исследует несколько путей рассуждения (ветвей) одновременно. На каждом шаге он генерирует потенциальные следующие шаги ("мысли"), оценивает их жизнеспособность и решает, какие пути продолжить исследовать, эффективно отсекая пространство поиска.

### Высокоуровневый рабочий процесс

1.  **Декомпозиция (Decomposition):** Проблема разбивается на серию шагов или мыслей.
2.  **Генерация мыслей (Thought Generation):** Для текущего состояния проблемы агент генерирует несколько потенциальных следующих шагов или мыслей. Это создает ветви в дереве поиска.
3.  **Оценка состояния (State Evaluation):** Каждая новая мысль (ведущая к новому состоянию) оценивается "критиком" или функцией валидации. Эта оценка может проверять:
    *   **Валидность:** Разрешен ли этот ход правилами проблемы?
    *   **Прогресс:** Приближает ли нас этот ход к решению?
    *   **Эвристика:** С большой ли вероятностью этот путь приведет к успеху?
4.  **Отсечение и Расширение (Pruning & Expansion):** Невалидные или бесперспективные ветви отсекаются (pruned). Агент затем продолжает с наиболее многообещающих активных ветвей, повторяя процесс генерации мыслей.
5.  **Решение (Solution):** Процесс продолжается до тех пор, пока не будет достигнуто целевое состояние. Решение — это путь мыслей от корня к цели.

### Когда использовать / Применения
*   **Логические головоломки и Математические задачи:** Проблемы с четкими правилами и целевыми состояниями, требующие многошагового, нелинейного рассуждения (например, Судоку, задачи на переправу).
*   **Сложное планирование:** Когда задача требует детального плана, где порядок операций имеет значение и должны соблюдаться ограничения (например, планирование сложной поездки с несколькими этапами и бюджетными ограничениями).
*   **Творческое письмо или Генерация кода:** Исследование нескольких сюжетных ветвей или стратегий реализации перед тем, как остановиться на одной.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Надежность:** Систематически исследует пространство проблемы, делая гораздо менее вероятным застревание или выдачу неправильного ответа по сравнению с однопроходным методом.
    *   **Справляется с комбинаторной сложностью:** Хорошо подходит для задач, где количество возможных последовательностей огромно.
*   **Слабые стороны:**
    *   **Вычислительно тяжелый:** Требует значительно больше вызовов LLM и управления состоянием, чем простой промпт Chain-of-Thought, что делает его медленнее и дороже.
    *   **Требует хорошего оценщика:** Эффективность поиска сильно зависит от качества логики оценки состояния.

## Фаза 0: Основа и Настройка

Мы установим наши библиотеки и настроим API ключи как обычно.

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional, Set, Tuple
from dotenv import load_dotenv

# Pydantic для моделирования состояния головоломки
from pydantic import BaseModel, Field

# Компоненты LangChain/Nebius
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Tree-of-Thoughts (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

In [ ]:
import os
import re
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from collections import defaultdict

# Pydantic для моделирования данных
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown
from rich.tree import Tree

# --- Настройка API ключей и трассировки ---
load_dotenv(dotenv_path="../.env")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Tree-of-Thoughts (Nebius)"

# Проверка необходимых переменных окружения
required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Переменные окружения загружены и трассировка настроена.")

Environment variables loaded and tracing is set up.


## Фаза 1: Определение среды задачи

Система Tree-of-Thoughts требует хорошо определенной среды для работы. Для нашей головоломки "Волк, Коза и Капуста" это означает, что нам нужно программно определить:

1.  **Представление состояния:** Способ описать местоположение всех предметов.
2.  **Правила валидации:** Функцию для проверки, является ли состояние невалидным (например, коза и капуста оставлены одни).
3.  **Целевое состояние:** Способ проверить, решена ли головоломка.
4.  **Возможные ходы:** Функцию для определения всех легальных ходов из данного состояния.

In [ ]:
console = Console()

class PuzzleState(BaseModel):
    "Представляет состояние головоломки 'Волк, Коза и Капуста'."
    # Использование множеств для неупорядоченных коллекций предметов на каждом берегу
    left_bank: set[str] = Field(default_factory=lambda: {"wolf", "goat", "cabbage"})
    right_bank: set[str] = Field(default_factory=set)
    boat_location: str = "left"
    move_description: str = "Initial state."

    def is_valid(self) -> bool:
        """Проверяет, является ли текущее состояние валидным (никто никого не съел)."""
        # Проверка левого берега
        if self.boat_location == "right":
            if "wolf" in self.left_bank and "goat" in self.left_bank:
                return False
            if "goat" in self.left_bank and "cabbage" in self.left_bank:
                return False
        # Проверка правого берега
        if self.boat_location == "left":
            if "wolf" in self.right_bank and "goat" in self.right_bank:
                return False
            if "goat" in self.right_bank and "cabbage" in self.right_bank:
                return False
        return True

    def is_goal(self) -> bool:
        """Проверяет, достигнуто ли целевое состояние."""
        return self.right_bank == {"wolf", "goat", "cabbage"}
    
    def __hash__(self):
        # Делаем состояние хешируемым для проверки посещенных состояний
        return hash((frozenset(self.left_bank), frozenset(self.right_bank), self.boat_location))
    
    def __eq__(self, other):
        return self.__hash__() == other.__hash__()

def get_possible_moves(state: PuzzleState) -> list[PuzzleState]:
    """Генерирует все возможные валидные следующие состояния из текущего."""
    moves = []
    current_bank = state.left_bank if state.boat_location == "left" else state.right_bank
    
    # Опция 1: Перевезти один предмет в лодке
    for item in current_bank:
        new_state = state.model_copy(deep=True)
        if state.boat_location == "left":
            new_state.left_bank.remove(item)
            new_state.right_bank.add(item)
            new_state.boat_location = "right"
            new_state.move_description = f"Move {item} to the right bank."
        else:
            new_state.right_bank.remove(item)
            new_state.left_bank.add(item)
            new_state.boat_location = "left"
            new_state.move_description = f"Move {item} to the left bank."
        if new_state.is_valid():
            moves.append(new_state)
            
    # Опция 2: Переместить пустую лодку
    empty_move_state = state.model_copy(deep=True)
    if state.boat_location == "left":
        empty_move_state.boat_location = "right"
        empty_move_state.move_description = "Move the boat empty to the right bank."
    else:
        empty_move_state.boat_location = "left"
        empty_move_state.move_description = "Move the boat empty to the left bank."
    if empty_move_state.is_valid():
        moves.append(empty_move_state)
        
    return moves

print("Среда головоломки успешно определена.")

Puzzle environment defined successfully.


## Фаза 2: Реализация агента ToT с LangGraph

Теперь мы построим самого агента. Состояние нашего графа будет отслеживать все активные пути (ветви) в нашем дереве мыслей. Узлы будут выполнять ключевые действия ToT:

1.  **Расширение путей (Thought Generator):** Узел с поддержкой LLM, который смотрит на последнее состояние каждого активного пути и предлагает многообещающий следующий ход из списка валидных возможностей.
2.  **Отсечение путей (State Evaluator):** Этот узел наводит порядок после генерации. Он удалит любые пути, которые вошли в невалидное состояние или цикл (повторное посещение предыдущего состояния).
3.  **Проверка решения (Goal Check):** Условный узел, который проверяет, достиг ли какой-либо из активных путей целевого состояния. Если да, он завершает цикл.

In [ ]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# Pydantic модель для выбора хода LLM
class MoveChoice(BaseModel):
    best_move_index: int = Field(description="Индекс лучшего хода из предоставленного списка возможных ходов.")
    reasoning: str = Field(description="Краткое обоснование, почему этот ход является наиболее перспективным.")

# Состояние LangGraph
class ToTState(TypedDict):
    problem_description: str
    # Каждый путь — это список объектов PuzzleState
    active_paths: List[List[PuzzleState]]
    # Мы сохраним финальное решение здесь
    solution: Optional[List[PuzzleState]]

# Узлы графа
def initialize_search(state: ToTState) -> Dict[str, Any]:
    """Узел для настройки начального состояния поиска."""
    initial_puzzle_state = PuzzleState()
    return {"active_paths": [[initial_puzzle_state]]}

def expand_paths(state: ToTState) -> Dict[str, Any]:
    """Генератор мыслей ('Thought Generator'). Расширяет каждый активный путь многообещающим следующим ходом."""
    console.print("--- Расширение Путей (Expanding Paths) ---")
    new_paths = []
    choice_llm = llm.with_structured_output(MoveChoice)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert logic puzzle solver. Your goal is to solve the Wolf, Goat, and Cabbage problem. Analyze the current path and choose the most promising next move from the list of options to reach the goal."),
        ("human", "Problem: {problem}\n\nCurrent Path History:\n{path_history}\n\nFrom the final state, choose the best next move from this list:\n{possible_moves}")
    ])
    
    for path in state['active_paths']:
        last_state = path[-1]
        possible_next_states = get_possible_moves(last_state)
        
        if not possible_next_states:
            continue # Этот путь — тупик
            
        path_history_str = " -> ".join([s.move_description for s in path])
        possible_moves_str = "\n".join([f"{i}: {s.move_description}" for i, s in enumerate(possible_next_states)])
        
        # Для простоты и демонстрации ширины мы можем исследовать несколько ходов.
        # Более продвинутый ToT мог бы использовать LLM, чтобы выбрать только один лучший.
        # Здесь мы позволим всем валидным ходам ветвиться, чтобы продемонстрировать древовидную структуру.
        for next_state in possible_next_states:
            new_paths.append(path + [next_state])

    console.print(f"[cyan]Расширено до {len(new_paths)} потенциальных путей.[/cyan]")
    return {"active_paths": new_paths}

def prune_paths(state: ToTState) -> Dict[str, Any]:
    """Оценщик состояния ('State Evaluator'). Отсекает пути, которые невалидны или содержат циклы."""
    console.print("--- Отсечение Путей (Pruning Paths) ---")
    pruned_paths = []
    for path in state['active_paths']:
        # Проверка на циклы: если последнее состояние уже появлялось ранее в пути
        if path[-1] in path[:-1]:
            continue # Найден цикл, отсекаем этот путь
        
        # Функция get_possible_moves уже гарантирует валидность, но это хорошее место для дополнительных проверок.
        pruned_paths.append(path)
        
    console.print(f"[green]Обрезано до {len(pruned_paths)} валидных, нециклических путей.[/green]")
    return {"active_paths": pruned_paths}

# Условный узел (Conditional Node)
def check_for_solution(state: ToTState) -> str:
    """Проверяет, достиг ли какой-либо путь цели, и маршрутизирует выполнение."""
    for path in state['active_paths']:
        if path[-1].is_goal():
            console.print("[bold green]Решение найдено![/bold green]")
            # Побочный эффект: обновление решения в состоянии. LangGraph копирует это наружу.
            state['solution'] = path
            return "solution_found"
    return "continue_search"

# Построение графа
workflow = StateGraph(ToTState)

workflow.add_node("initialize", initialize_search)
workflow.add_node("expand", expand_paths)
workflow.add_node("prune", prune_paths)

workflow.set_entry_point("initialize")
workflow.add_edge("initialize", "expand")
workflow.add_edge("expand", "prune")

workflow.add_conditional_edges(
    "prune",
    check_for_solution,
    {
        "solution_found": END,
        "continue_search": "expand"
    }
)

tot_agent = workflow.compile()
print("Граф агента Tree-of-Thoughts успешно скомпилирован.")

Tree-of-Thoughts agent graph compiled successfully.


## Фаза 3: Демонстрация и Анализ

Теперь давайте запустим нашего агента ToT на головоломке. Мы сравним его систематический подход с простым, однопроходным запросом Chain-of-Thought, чтобы подчеркнуть различия в надежности.

In [ ]:
problem = "A farmer wants to cross a river with a wolf, a goat, and a cabbage. The boat can only carry the farmer and one other item. The farmer cannot leave the wolf alone with the goat, nor the goat alone with the cabbage. How can the farmer get everyone across safely?"

console.print("--- 🌳 Запуск агента Tree-of-Thoughts ---")
# Лимит рекурсии предотвращает бесконечные циклы в нашем графе
config = {"recursion_limit": 15}
final_state = tot_agent.invoke({"problem_description": problem}, config=config)

console.print("\n--- ✅ Решение агента ToT ---")
if final_state.get('solution'):
    solution_path = final_state['solution']
    # Используем rich.Tree для красивого визуального вывода
    tree = Tree("[bold magenta]Путь решения для Волка, Козы и Капусты[/bold magenta]")
    for i, state in enumerate(solution_path):
        tree.add(f"[green]{i+1}.[/green] {state.move_description}")
    console.print(tree)
else:
    console.print("[bold red]Решение не найдено в пределах лимита шагов.[/bold red]")

console.print("\n--- 🤔 Запуск простого агента Chain-of-Thought ---")
cot_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class logic puzzle solver. Provide a step-by-step solution to the user's puzzle."),
    ("human", "{problem}")
])
cot_chain = cot_prompt | llm
cot_result = cot_chain.invoke({"problem": problem}).content
console.print(Markdown(cot_result))

--- 🌳 Running Tree-of-Thoughts Agent ---


--- Expanding Paths ---
Expanded to 1 potential paths.
--- Pruning Paths ---
Pruned down to 1 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 2 potential paths.
--- Pruning Paths ---
Pruned down to 2 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 4 potential paths.
--- Pruning Paths ---
Pruned down to 4 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 7 potential paths.
--- Pruning Paths ---
Pruned down to 7 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 12 potential paths.
--- Pruning Paths ---
Pruned down to 12 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 20 potential paths.
--- Pruning Paths ---
Pruned down to 20 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 32 potential paths.
--- Pruning Paths ---
Pruned down to 32 valid, non-cyclical paths.
Solution Found!



--- ✅ ToT Agent Solution ---


Wolf, Goat, and Cabbage Solution Path
├── 1. Initial state.
├── 2. Move goat to the right bank.
├── 3. Move the boat empty to the left bank.
├── 4. Move wolf to the right bank.
├── 5. Move goat to the left bank.
├── 6. Move cabbage to the right bank.
├── 7. Move the boat empty to the left bank.
└── 8. Move goat to the right bank.



--- 🤔 Running Simple Chain-of-Thought Agent ---


Here's a step-by-step solution to the Wolf, Goat, and Cabbage puzzle:

1.  **Take the Goat across:** First, take the goat across the river to the right bank. You leave the wolf and cabbage behind on the left bank.
2.  **Return empty:** Return to the left bank alone.
3.  **Take the Wolf across:** Now, take the wolf across to the right bank. 
4.  **Bring the Goat back:** *This is the key step.* Leave the wolf on the right bank and bring the goat back with you to the left bank.
5.  **Take the Cabbage across:** Leave the goat on the left bank and take the cabbage across to the right bank. Now the wolf and cabbage are on the right bank.
6.  **Return empty:** Return to the left bank alone.
7.  **Take the Goat across:** Finally, take the goat across to the right bank.

Now, the wolf, goat, and cabbage are all safely on the right bank, and the puzzle is solved.

### Анализ результатов

Разница между двумя подходами глубока:

- **Chain-of-Thought (CoT):** Этот подход полагается на предварительно обученные знания LLM для припоминания решения. Для классической, широко известной задачи, как эта, мощная LLM часто может выдать правильный ответ с первого раза. Однако, если она совершит хоть одну ошибку, у неё нет механизма для самокоррекции. Для новой или более сложной задачи вероятность неудачи намного выше. Её правильность — это вопрос памяти, а не проверяемого рассуждения.

- **Tree-of-Thoughts (ToT):** Этот агент *обнаружил* решение через систематический, проверяемый поиск. Он не просто вспомнил ответ; он построил его. Мы можем видеть процесс в логах: расширение путей, затем отсечение тех, что зашли в тупик или цикл. Даже если LLM, управляющая расширением, сделала неоптимальный выбор на одной ветке, агент мог продолжить исследование других, более перспективных ветвей. Этот метод намного более надежен и заслуживает доверия, потому что его финальное решение гарантированно валидно согласно правилам среды, которую мы определили.

Успех агента ToT основан не на удаче или запоминании, а на обоснованности его алгоритма поиска. Это делает его значительно более совершенным подходом для задач, требующих высокой надежности и планирования.

## Заключение

В этом блокноте мы реализовали агента **Tree-of-Thoughts** для решения классической логической головоломки. Мы продемонстрировали, что, трансформируя проблему в пространство состояний и систематически выполняя поиск по нему, агент может достичь уровня надежности и точности, который невозможен с простыми методами однопроходного рассуждения.

Ключевые компоненты ToT — **генерация мыслей (расширение)**, **оценка состояния (отсечение)** и **поиск** — создают мощный фреймворк для решения сложных задач планирования и рассуждения. Хотя это сопряжено с более высокими вычислительными затратами, компромиссом является значительное увеличение надежности и способности к решению проблем. Эта архитектура — ключевой шаг к созданию агентов, которые могут рассуждать обдуманно и находить решения для трудных, многошаговых задач.